In [1]:
import blocks as bl
import constants as cst
import elementDefs as ed
import pyfe3Dgcl as p3g
import numpy as np

In [2]:
'''Standard test setup'''
data = cst.CAD_DATA
lgl_infs = [4, 4.5, 5, 6] #preparing the k infinity values
load_case = cst.LOAD_C[2]#landing load case used in the sensitivity study
#load_case["FT"] = 5000 #landing at full thrust - a weird load case that tests everything at once
eleDict = ed.eledict(cst.CONSTS, cst.INITIAL, cst.CODES)
meshOut = bl.mesh_block(data, cst.INITIAL, eleDict, cst.CONSTS, cst.CODES)

In [3]:
def beam_mass(beamEle:p3g.BeamCWithProp):
    return beamEle.length*beamEle.beamprop.intrho

def quad_mass(quadEle:p3g.QuadWithProp, ncoords, nid_pos):
    coords = lambda nx:ncoords[nid_pos[nx], :]
    coords1 = coords(quadEle.n1)
    coords2 = coords(quadEle.n2)
    coords3 = coords(quadEle.n3)
    coords4 = coords(quadEle.n4)

    area = .5*(np.linalg.norm(np.cross(coords4-coords1, coords2-coords1))+ 
               np.linalg.norm(np.cross(coords2-coords3, coords4-coords3)))
    return cst.CONSTS["RHO_ALU"]*cst.INITIAL["tskin"]*area

In [4]:
beam_tot_masses = dict()
for beam in meshOut["elements"]["beam"]:
    if beam.eleid not in beam_tot_masses.keys():
        beam_tot_masses[beam.eleid]=0.
    beam_tot_masses[beam.eleid]+=beam_mass(beam)

quad_tot_mass = 0
for quad in meshOut["elements"]["quad"]:
    quad_tot_mass+=quad_mass(quad, meshOut["ncoords"], meshOut["nid_pos"])

In [6]:
totSizeableMmass = quad_tot_mass+sum(beam_tot_masses.values())
totSizeableWeight = cst.CONSTS["G0"]*totSizeableMmass
print(f"skin mass: {quad_tot_mass} kg")
print("Masses of beam elements:")
for key, val in beam_tot_masses.items():
    print(f"{key}: {val} kg") 
print(f"\nTotal mass: {totSizeableMmass} kg")
print(f"Total weight: {totSizeableWeight} N")

skin mass: 884.6232409966718 kg
Masses of beam elements:
rl: 183.47311138942123 kg
rb: 148.27382267218982 kg
lt: 180.79345304128867 kg
lg: 88.07914158969398 kg
sp: 739.0475821023846 kg

Total mass: 2224.29035179165 kg
Total weight: 21820.28835107609 N


Weight from pure inertia

In [8]:
mesh = meshOut["mesh"]
tot_inertia_mass = sum([ine.mn for ine in mesh.inertia])
totmass = totSizeableMmass+tot_inertia_mass
totweight = cst.CONSTS["G0"]*totmass
print(f"Total mass from inertia: {tot_inertia_mass}")
print(f"total mass: {totmass}")
print(f"Total weight: {totweight}")
print(f"tot_weight_with_load_factor: {4.25*totweight}")

Total mass from inertia: 23655.999999999694
total mass: 25880.290351791344
Total weight: 253885.6483510731
tot_weight_with_load_factor: 1079014.0054920607
